# Tool or Function Calling
To build agent workflows, llms need access to tools. Some llms are not able to handle tools - llama3.2 is able to handle these so we will use it for this demo.\
Example workflow: query -> llm identifies what tool and what inputs are needed -> feed into tool -> receive and parse output

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model='llama3.2', base_url='http://localhost:11434')

### Tool Creation
Langchain web search tools: https://python.langchain.com/docs/integrations/tools/ \
In this example, we'll use tavily to search: https://app.tavily.com/home. Can also make your own search, but will be much more intensive.

Use the langchain tool decorator to turn any function into a tool. The function must have a description and Args for the llm to understand.

In [4]:
### Tool Creation

# allows you to convert any python function to a tool
from langchain_core.tools import tool

@tool
def add(a, b):
    """
    Adds two integer numbers together

    Args:
    a: First Integer
    b: Second Integer
    """
    return a + b

@tool
def multiply(a, b):
    """
    Multiply two integer numbers together

    Args:
    a: First Integer
    b: Second Integer
    """

We can see that this causes the function 'add' to be of type StructuredTool. This can then be added to a chain as a Runnable and can be invoked.

In [5]:
add

StructuredTool(name='add', description='Adds two integer numbers together\n\nArgs:\na: First Integer\nb: Second Integer', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x00000222E5B480E0>)

In [7]:
add.name, add.description, add.args, add.args_schema.model_json_schema()

('add',
 'Adds two integer numbers together\n\nArgs:\na: First Integer\nb: Second Integer',
 {'a': {'title': 'A'}, 'b': {'title': 'B'}},
 {'description': 'Adds two integer numbers together\n\nArgs:\na: First Integer\nb: Second Integer',
  'properties': {'a': {'title': 'A'}, 'b': {'title': 'B'}},
  'required': ['a', 'b'],
  'title': 'add',
  'type': 'object'})

### Invoking tools in a chain
Since tools are no longer plain functions, they must be invoked instead of called. This is to help prepare them for langchain implementation so the llm can properly call the tools.

In [8]:
add.invoke({'a': 1, 'b': 2})

3

### Bind tools to an llm

In [10]:
tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)
llm_with_tools

RunnableBinding(bound=ChatOllama(model='llama3.2', base_url='http://localhost:11434'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Adds two integer numbers together\n\nArgs:\na: First Integer\nb: Second Integer', 'parameters': {'properties': {'a': {}, 'b': {}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply two integer numbers together\n\nArgs:\na: First Integer\nb: Second Integer', 'parameters': {'properties': {'a': {}, 'b': {}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

Here the llm does not return the answer (content=''), but it instead shows what the name of the tool we should use is (see tool_calls) along with the arguments that it will take in. In this case, the ToolCall will have name='add' and arguments correctly passed.

In [13]:
question = "What is 1 plus 2? Also, what is 2 times 14?"
llm_with_tools.invoke(question).tool_calls

[{'name': 'add',
  'args': {'a': '1', 'b': '2'},
  'id': 'e842fdc5-fdd0-4425-9985-45851a0aef34',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': '2', 'b': '14'},
  'id': '569a813e-4dc9-420c-b8f4-a29f9a32f1d7',
  'type': 'tool_call'}]

# Web Search Tools
For this demo, we'll create a few tools that can perform internet searches. Searches will return articles or web text that can then be fed into an LLM as context. Then, we'll combine the tools and show how to use them with an LLM.

## DuckDuckGoSearch
https://python.langchain.com/docs/integrations/tools/ddg/

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.invoke("Summarize the S&P500 news for today, including the current date.")

"S&P 500 Today: Get all information on the S&P 500 Index including historical chart, news and constituents. ... Date Analyst stock Rating Price 4/17/2025 ... The S&P 500 fell 1.5% during the shortened trading week—markets are closed tomorrow in observation of Good Friday—while the Dow and Nasdaq Composite gave up 2.7% and 2.6%, respectively. S&P 500 ekes out gain; Dow and Nasdaq close lower The S&P 500 inched 0.1% higher on Thursday, snapping a two-day losing streak. The Dow and Nasdaq Composite lost 1.3% and 0.1%, respectively, in ... The S&P 500 rose nearly 0.1% and managed to eke out a gain of about 0.5% for the week. The tech-heavy Nasdaq Composite (^IXIC) rose 0.5% and also climbed in positive territory for the week. Get S&P 500 Index End of day (.SP500) real-time stock quotes, news, price and financial information from Reuters to inform your trading and investments ... Today's Range 5,275.70 5,275.70 ..."

## Tavily Search
Tavily Sarch has its own package now in langchain-tavily instead of in community.tools. For this exercise we'll use the old one though. Note: 1000 searches per month are available on free plan of Tavily. \
https://python.langchain.com/docs/integrations/tools/tavily_search/ \
https://docs.tavily.com/documentation/api-reference/endpoint/search

In [13]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
    max_results=5,
    search_depth='advanced',
    include_answer=True,
    include_raw_content=True
)

In [15]:
results = search.invoke("Summarize the S&P500 news for today, including the current date.")
len(results)

5

## Wikipedia Search
Returns pages to be fed to llm as context based on question. \
https://python.langchain.com/docs/integrations/tools/wikipedia/

In [9]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
question = 'What is the capital of UAE?'
response = wikipedia.invoke(question)

In [16]:
print(response)

Page: United Arab Emirates
Summary: The United Arab Emirates (UAE), or simply the Emirates, is a country in West Asia, in the Middle East, at the eastern end of the Arabian Peninsula. It is a federal elective monarchy made up of seven emirates, with Abu Dhabi serving as its capital. It shares land borders with Oman to the east and northeast, and with Saudi Arabia to the southwest; as well as maritime borders in the Persian Gulf with Qatar and Iran, and with Oman in the Gulf of Oman. As of 2024, the UAE has an estimated population of over 10 million, of which 11% are Emiratis; Dubai is its most populous city and is an international hub. Islam is the official religion and Arabic is the official language, while English is the most spoken language and the language of business.
The United Arab Emirates oil and natural gas reserves are the world's seventh and seventh-largest, respectively. Zayed bin Sultan Al Nahyan, ruler of Abu Dhabi and the country's first president, oversaw the developme

## PubMed
More than 35M citations for biomed literature from scientific sources. May contain full text content from PubMed Central and other publisher websites.
https://python.langchain.com/docs/integrations/tools/pubmed/

In [18]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun
tool = PubmedQueryRun()
print(tool.invoke("What causes lung cancer?"))

Too Many Requests, waiting for 0.20 seconds...
Published: 2025-04-23
Title: 4-fluorophenylacetamide acetyl coumarin induces pro-inflammatory M1 macrophage polarization and suppresses the immunosuppressive M2 phenotype through PI3k/AKT/NF-κB modulation.
Copyright Information: © 2025. The Author(s), under exclusive licence to Springer Nature B.V.
Summary::
BACKGROUND: The tumor microenvironment plays a critical role in cancer progression, with tumor-associated macrophages regulating immune responses. These macrophages can adopt a pro-inflammatory M1 phenotype that suppresses tumor growth or an anti-inflammatory M2 phenotype that promotes progression. Reprogramming macrophages toward the M1 phenotype is a therapeutic strategy. Previous studies showed that 4-Fluorophenylacetamide-acetyl coumarin (4-FPAC), a synthetic coumarin derivative, exhibits cytostatic activity in A549 lung carcinoma cells by modulating reactive oxygen species (ROS), nitric oxide synthase, and signaling pathways, incl

# Tool Calling with LLM 
Integrating the two previous sections, we'll use a search and send the result to LLM as context. Let's first set up all the different search tools:

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools import TavilySearchResults
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool

@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information.

    Args:
    query: The search query
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wikipedia.invoke(query)

@tool
def pubmed_search(query):
    """
    Search pubmed for medical and life science queries.

    Args:
    query: The search query
    """
    pubmed = PubmedQueryRun()
    return pubmed.invoke(query)

@tool
def tavily_search(query):
    """
    Search the web for realtime and latest news and information.
    For example, news, stock market, weather updates, etc.

    Args:
    query: The search query
    """
    search = TavilySearchResults(
    max_results=5,
    search_depth='advanced',
    include_answer=True,
    include_raw_content=True
    )
    return search.invoke(query)


### Combine tools for LLM to use

In [7]:
tools = [wikipedia_search, pubmed_search, tavily_search]

# struct format for tool list
list_of_tools = { tool.name: tool for tool in tools }
list_of_tools

{'wikipedia_search': StructuredTool(name='wikipedia_search', description='Search wikipedia for general information.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.wikipedia_search'>, func=<function wikipedia_search at 0x000001506F7A1D00>),
 'pubmed_search': StructuredTool(name='pubmed_search', description='Search pubmed for medical and life science queries.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.pubmed_search'>, func=<function pubmed_search at 0x000001506E28F560>),
 'tavily_search': StructuredTool(name='tavily_search', description='Search the web for realtime and latest news and information.\nFor example, news, stock market, weather updates, etc.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.tavily_search'>, func=<function tavily_search at 0x000001506E28CD60>)}

### Bind to LLM

In [17]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

llm = ChatOllama(model='llama3.2', base_url='http://localhost:11434')

In [10]:
llm_with_tools = llm.bind_tools(tools)

In [11]:
query = "What is the latest news?"

response = llm_with_tools.invoke(query)

There is no content here again, but it is calling the tavily_search function to search the web for latest news. Rerunning with a medical-related question shows that the tool would call pubmed_search instead, since we defined that tool to handle medical information.

In [16]:
print(response)
print('\n')
print(response.tool_calls)

content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-04-23T20:47:35.6957206Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8979738800, 'load_duration': 7055983200, 'prompt_eval_count': 297, 'prompt_eval_duration': 1562000000, 'eval_count': 20, 'eval_duration': 359000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='tavily_search', arguments={'query': 'latest news'}))])} id='run-7281916d-207f-4f9d-a599-fc8d1c6c39a0-0' tool_calls=[{'name': 'tavily_search', 'args': {'query': 'latest news'}, 'id': 'f6cbd162-712c-4cc9-bf35-b1de9ad868a6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 297, 'output_tokens': 20, 'total_tokens': 317}


[{'name': 'tavily_search', 'args': {'query': 'latest news'}, 'id': 'f6cbd162-712c-4cc9-bf35-b1de9ad868a6', 'type': 'tool_call'}]


# Using an LLM to get the FINAL RESULT
So far, we've been able to use an LLM to identify which tool to use and specify the input parameters. Now, we should use this determined input and tool to actually generate a response based on the user's query.

Initial tool call (as done previously) to grab the input params and desired tool:

In [28]:
query = "What new findings have been published on skin cancer?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-23T21:06:44.5920272Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1435877400, 'load_duration': 16650700, 'prompt_eval_count': 301, 'prompt_eval_duration': 7000000, 'eval_count': 21, 'eval_duration': 1411000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='pubmed_search', arguments={'query': 'skin cancer new findings'}))])}, id='run-3653c871-8652-4858-8915-86e072e322c1-0', tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'skin cancer new findings'}, 'id': '714923a5-175f-4160-8b17-07fda5254624', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'output_tokens': 21, 'total_tokens': 322})

Then, we should append this to messages to be fed back into an llm. We have identified human messages and ai messages, so the LLM will be able to distinguish between these.

In [29]:
messages.append(ai_msg)
messages

[HumanMessage(content='What new findings have been published on skin cancer?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-23T21:06:44.5920272Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1435877400, 'load_duration': 16650700, 'prompt_eval_count': 301, 'prompt_eval_duration': 7000000, 'eval_count': 21, 'eval_duration': 1411000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='pubmed_search', arguments={'query': 'skin cancer new findings'}))])}, id='run-3653c871-8652-4858-8915-86e072e322c1-0', tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'skin cancer new findings'}, 'id': '714923a5-175f-4160-8b17-07fda5254624', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'output_tokens': 21, 'total_tokens': 322})]

Next, for each tool that will be used, we should append the tool message to messages as well. 

In [30]:
for tool_call in ai_msg.tool_calls:
    name = tool_call['name'].lower()
    selected_tool = list_of_tools[name] # dict of tools and their related StructuredTool
    tool_msg = selected_tool.invoke(tool_call)

    messages.append(tool_msg)

Too Many Requests, waiting for 0.20 seconds...


In [31]:
messages

[HumanMessage(content='What new findings have been published on skin cancer?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-23T21:06:44.5920272Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1435877400, 'load_duration': 16650700, 'prompt_eval_count': 301, 'prompt_eval_duration': 7000000, 'eval_count': 21, 'eval_duration': 1411000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='pubmed_search', arguments={'query': 'skin cancer new findings'}))])}, id='run-3653c871-8652-4858-8915-86e072e322c1-0', tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'skin cancer new findings'}, 'id': '714923a5-175f-4160-8b17-07fda5254624', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'output_tokens': 21, 'total_tokens': 322}),
 ToolMessage(content='Published: 2025-04-22\nTitle: [Melanoma prevention - to screen

Note: looking at above output, we see that:
- First, the llm chose pubmed_search as the tool and 'skin cancer new findings' as the query. This was extracted from our original query of 'What new findings have been published on skin cancer'. 
- Then, the chosen tool is called, which brings back some context on recent articles related to skin cancer. 
- Finally, the original query is evaluated using this context and a response is generated.
### Finally, calling the entire llm with tools, we can get some news related to our query:

In [32]:
response = llm_with_tools.invoke(messages)
print(response.content)

Based on the latest research published in various medical journals, here are some new findings on skin cancer:

1. **Early detection through 3D whole-body photography**: A new study has shown that 3D whole-body photography, combined with computer-based AI-assisted risk assessment of digital dermoscopic images, can significantly improve skin cancer screening, particularly in high-risk patients.
2. **Targeted educational campaigns**: Researchers recommend targeted educational campaigns among risk groups, including men and people with lower socioeconomic status, to increase knowledge about skin cancer-associated factors and encourage participation in skin cancer screening programs.
3. **Increased detection of thin melanomas**: Systematic skin cancer screening has been shown to detect more melanomas, particularly thinner ones, which have a better prognosis when diagnosed early.
4. **No evidence of decreased mortality**: Despite increased detection of melanomas through screening, there is c